Processing notebook for COVID datasets to be used in a Tableau dashboard

Sources

1) https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data/vbim-akqf/about_data
Filtered on the CDC website by:
- sex - not one of 'Unknown', 'Missing', 'NA', 'Other'
- age_group - not one of 'Missing', 'NA'
- race_ethnicity_combined - not one of 'Unknown', 'Missing', 'NA'
- Remove all rows other than ['cdc_report_dt', 'sex', 'age_group', 'race_ethnicity_combined',
       'hosp_yn', 'icu_yn', 'death_yn', 'medcond_yn']
However, you could also filter out any rows with 'Unknown' or 'Missing' in the 'hosp_yn', 'icu_yn', 'death_yn' and not include the 'medcond_yn' col
5.57gb

2) https://data.cdc.gov/Case-Surveillance/Weekly-United-States-COVID-19-Cases-and-Deaths-by-/pwn4-m3yp/about_data
612kb
Filter on CDC website by:
- Drop all columsn other than ['state', 'start_date', 'new_cases', 'new_deaths']



In [2]:
import pandas as pd

Processing for the case dataset. Since this was such a large dataset, I'm going to have to pare the dataset down. I didn't get a super clear answer of what was the max size of data that can be used in Tableau Public, with some sources saying <1gb and then after using the dataset, the app also saying less than 15m rows.

In [4]:
# Load in case data

df = pd.read_csv('COVID-19_Case_Surveillance_Public_Use_Data_20240718.csv')

In [5]:
# Select columns

df = df[['cdc_report_dt', 'sex', 'age_group', 'race_ethnicity_combined',
       'hosp_yn', 'icu_yn', 'death_yn', 'medcond_yn']]

In [6]:
# Change column names to be more clear

df.columns = ['cdc_report_dt', 'sex', 'age_group', 'race_ethnicity',
       'hospitalized', 'icu', 'death', 'preexisting_medcondition']

len(df)

70701797

In [7]:
df.head(10)

,cdc_report_dt,sex,age_group,race_ethnicity,hospitalized,icu,death,preexisting_medcondition
0,2022/01/31,Female,20 - 29 Years,"Asian, Non-Hispanic",Missing,Missing,Missing,Missing
1,2020/07/11,Female,20 - 29 Years,"Asian, Non-Hispanic",Unknown,Unknown,Unknown,No
2,2022/09/01,Female,20 - 29 Years,"Asian, Non-Hispanic",Unknown,Missing,No,Missing
3,2023/01/10,Female,20 - 29 Years,"Asian, Non-Hispanic",Unknown,Unknown,Missing,Unknown
4,2020/11/01,Female,20 - 29 Years,"Asian, Non-Hispanic",Unknown,Missing,Unknown,Missing
5,2024/06/24,Female,20 - 29 Years,"Asian, Non-Hispanic",Unknown,Missing,Unknown,Missing
6,2022/10/01,Female,20 - 29 Years,"Asian, Non-Hispanic",Missing,Missing,Missing,Missing
7,2022/07/08,Female,20 - 29 Years,"Asian, Non-Hispanic",No,Missing,Missing,Missing
8,2021/09/08,Female,20 - 29 Years,"Asian, Non-Hispanic",No,Missing,No,Missing
9,2022/01/21,Female,20 - 29 Years,"Asian, Non-Hispanic",No,Unknown,Missing,Yes


In [11]:
# Get year and month columns, first by removing NaN entries from 'cdc_report_dt'

df = df[df['cdc_report_dt'].notnull()]

len(df)

64744531

In [13]:
# Get year/month columns from 'cdc_report_dt'

def find_year(s):
    return int(s[:4])

def find_month(s):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    month_number = int(s[5:7])
    return months[month_number-1]

In [14]:
df['year'] = df['cdc_report_dt'].apply(find_year)

df.year.value_counts()

year
2022    26239736
2021    19684148
2020     8068261
2023     6556838
2024     4195548
Name: count, dtype: int64

In [15]:
df['month'] = df['cdc_report_dt'].apply(find_month)

df.month.value_counts()

month
Jan    13167888
Dec     8957772
Feb     5172326
Jun     4839285
Nov     4817338
Sep     4755668
Aug     4543409
Mar     4456959
Oct     3899482
May     3448534
Jul     3357687
Apr     3328183
Name: count, dtype: int64

In [61]:
# save initial file, will need to slice more off
df.to_csv('covid_case_data_20240718_full.csv', index=False)

In [16]:
# drop more unneeded columns

df.drop(['cdc_report_dt', 'preexisting_medcondition', 'sex'], axis=1, inplace=True)

df.race_ethnicity.value_counts()

race_ethnicity
White, Non-Hispanic                                     36620322
Hispanic/Latino                                         14038653
Black, Non-Hispanic                                      8661624
Asian, Non-Hispanic                                      2446178
Multiple/Other, Non-Hispanic                             2160177
American Indian/Alaska Native, Non-Hispanic               668503
Native Hawaiian/Other Pacific Islander, Non-Hispanic      149074
Name: count, dtype: int64

In [68]:
df[['age_group', 'race_ethnicity', 'hospitalized', 'icu', 'death', 'month']].describe()

,age_group,race_ethnicity,hospitalized,icu,death,month
count,64742216,64742216,64742216,64742216,64742216,64742216
unique,9,7,4,4,4,12
top,20 - 29 Years,"White, Non-Hispanic",Missing,Missing,Missing,Jan
freq,10633145,36619285,27136853,54261309,29943644,13167231


In [69]:
# save working file, but it's still too large at 3.9gb
df.to_csv('covid_case_data_20240718.csv', index=False)

In [17]:
# will try to make the file size smaller by truncating some of the text

df.age_group.value_counts()

age_group
20 - 29 Years    10633821
30 - 39 Years    10337565
40 - 49 Years     8918956
50 - 59 Years     8478110
10 - 19 Years     7434446
60 - 69 Years     6692122
0 - 9 Years       5039337
70 - 79 Years     4294017
80+ Years         2916157
Name: count, dtype: int64

In [18]:
df['age_group'] = df['age_group'].apply(lambda s: s.replace('Years','').replace(' ',''))

df.age_group.value_counts()

age_group
20-29    10633821
30-39    10337565
40-49     8918956
50-59     8478110
10-19     7434446
60-69     6692122
0-9       5039337
70-79     4294017
80+       2916157
Name: count, dtype: int64

In [19]:
df.race_ethnicity.value_counts()

race_ethnicity
White, Non-Hispanic                                     36620322
Hispanic/Latino                                         14038653
Black, Non-Hispanic                                      8661624
Asian, Non-Hispanic                                      2446178
Multiple/Other, Non-Hispanic                             2160177
American Indian/Alaska Native, Non-Hispanic               668503
Native Hawaiian/Other Pacific Islander, Non-Hispanic      149074
Name: count, dtype: int64

In [20]:
df['race_ethnicity'] = df['race_ethnicity'].apply(lambda s: s.replace(', Non-Hispanic', ''))

df.race_ethnicity.value_counts()

race_ethnicity
White                                     36620322
Hispanic/Latino                           14038653
Black                                      8661624
Asian                                      2446178
Multiple/Other                             2160177
American Indian/Alaska Native               668503
Native Hawaiian/Other Pacific Islander      149074
Name: count, dtype: int64

In [22]:
def replace_hospitalized_icu_death_values_to_bool(s):
    if s in ['Missing', 'Unknown']:
        return None
    elif s == 'Yes':
        return True
    else:
        return False

df['hospitalized'] = df['hospitalized'].apply(replace_hospitalized_icu_death_values_to_bool)
df['icu'] = df['icu'].apply(replace_hospitalized_icu_death_values_to_bool)
df['death'] = df['death'].apply(replace_hospitalized_icu_death_values_to_bool)

In [89]:
# still 1.8 gb
df.to_csv('covid_case_data_20240718.csv', index=False)

In [ ]:
# removing all values with missing data in the three columns: hospitalized, icu, death

In [23]:
len(df)

64744531

In [24]:
df.hospitalized.value_counts(dropna=False)

hospitalized
None     38058369
False    24045696
True      2640466
Name: count, dtype: int64

In [25]:
df.dropna(axis=0, how='all', subset=['hospitalized','icu','death'], inplace=True)

len(df)

33899801

In [102]:
# 1.1gb...
df.to_csv('covid_case_data_20240718.csv', index=False)

In [26]:
# since the state dataset only goes from 1/2020-5/2023, we'll remove anything beyond that

df2 = df[df['year']==2023]
indices = df2[df2['month'].isin(['May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])].index

df.drop(indices, inplace=True)

df.year.value_counts(dropna=False)

year
2021    11784651
2022    11591679
2020     5721511
2024     1829967
2023     1490896
Name: count, dtype: int64

In [148]:
# 1.05 gb, getting there
df.to_csv('covid_case_data_20240718.csv', index=False)

In [27]:
# truncate race/enthnicity column names

df.race_ethnicity.value_counts(dropna=False)

race_ethnicity
White                                     19118362
Hispanic/Latino                            6935268
Black                                      3934111
Asian                                      1076207
Multiple/Other                              974343
American Indian/Alaska Native               303574
Native Hawaiian/Other Pacific Islander       76839
Name: count, dtype: int64

In [28]:
def truncate_race_names(s):
    if s == 'Hispanic/Latino':
        return 'Hispanic'
    elif s in ['Multiple/Other',
               'American Indian/Alaska Native',
               'Native Hawaiian/Other Pacific Islander']:
        return 'Other'
    else:
        return s

df['race_ethnicity'] = df['race_ethnicity'].apply(truncate_race_names)

df.race_ethnicity.value_counts(dropna=False)

race_ethnicity
White       19118362
Hispanic     6935268
Black        3934111
Other        1354756
Asian        1076207
Name: count, dtype: int64

In [157]:
df.to_csv('covid_case_data_20240718.csv', index=False)

In [ ]:
# so apparently, i can only have <15mil rows in Tableau public, so we have to reduce more
# this time by removing any missing/null data in 'hospitalized', 'icu' or 'death' columns

In [29]:
len(df)

32418704

In [35]:
df.dropna(subset = ['hospitalized','icu','death'], inplace=True)

len(df)

1712832

In [6]:
df.to_csv('covid_case_data_20240718_reduced.csv', index=False)

Processing for the weekly state dataset. This shouldn't have size issues, but will have to clean some date columns and aggregate by state+month

In [37]:
# Load in state/time data

df = pd.read_csv('Weekly_United_States_COVID-19_Cases_and_Deaths_by_State_-_ARCHIVED_20240720.csv')

df.head()

,date_updated,state,start_date,end_date,tot_cases,new_cases,tot_deaths,new_deaths,new_historic_cases,new_historic_deaths
0,02/23/2023,AZ,02/16/2023,02/22/2023,2434631,3716,33042,39,23150,0
1,12/22/2022,LA,12/15/2022,12/21/2022,1507707,4041,18345,21,21397,0
2,02/23/2023,GA,02/16/2023,02/22/2023,3061141,5298,42324,88,6800,0
3,03/30/2023,LA,03/23/2023,03/29/2023,1588259,2203,18858,23,5347,0
4,02/02/2023,LA,01/26/2023,02/01/2023,1548508,5725,18572,47,4507,0


In [38]:
# drop unnecessary columns

df.drop(['date_updated', 'end_date', 'tot_cases', 'tot_deaths', 'new_historic_cases', 'new_historic_deaths'], axis=1, inplace=True)

df.head()

,state,start_date,new_cases,new_deaths
0,AZ,02/16/2023,3716,39
1,LA,12/15/2022,4041,21
2,GA,02/16/2023,5298,88
3,LA,03/23/2023,2203,23
4,LA,01/26/2023,5725,47


In [39]:
# we'll need to convert start_date to year and month columns

def find_year2(s):
    return int(s[6:])

def find_month2(s):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    month_number = int(s[:2])
    return months[month_number-1]

df['year'] = df['start_date'].apply(find_year2)
df['month'] = df['start_date'].apply(find_month2)

df.drop('start_date', axis=1, inplace=True)

df.head()

,state,new_cases,new_deaths,year,month
0,AZ,3716,39,2023,Feb
1,LA,4041,21,2022,Dec
2,GA,5298,88,2023,Feb
3,LA,2203,23,2023,Mar
4,LA,5725,47,2023,Jan


In [185]:
# groupby state/year/month, aggregate sums

df = (
    df.groupby(['state','year','month'])
        .agg(new_cases=('new_cases', 'sum'), new_deaths=('new_deaths', 'sum'))
)

In [188]:
df.reset_index(inplace=True)

In [189]:
df

,state,year,month,new_cases,new_deaths
0,AK,2020,Apr,225,7
1,AK,2020,Aug,1988,13
2,AK,2020,Dec,14012,97
3,AK,2020,Feb,0,0
4,AK,2020,Jan,0,0
...,...,...,...,...,...
2455,WY,2023,Apr,542,11
2456,WY,2023,Feb,1156,23
2457,WY,2023,Jan,835,21
2458,WY,2023,Mar,1084,18


In [8]:
df = pd.read_csv('covid_state_data_20240720.csv')

In [10]:
def convert_to_year_month_string(y, m):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    month_num = months.index(m) + 1
    if month_num < 10:
        month_str = '0' + str(month_num)
    else:
        month_str = str(month_num)
    return f'{y}-{month_str}'

In [12]:
convert_to_year_month_string(2020, 'Dec')

'2020-12'

In [13]:
df['year-month'] = df.apply(lambda d: convert_to_year_month_string(d.year, d.month), axis=1)

In [14]:
df

,state,year,month,new_cases,new_deaths,year-month
0,AK,2020,Apr,225,7,2020-04
1,AK,2020,Aug,1988,13,2020-08
2,AK,2020,Dec,14012,97,2020-12
3,AK,2020,Feb,0,0,2020-02
4,AK,2020,Jan,0,0,2020-01
...,...,...,...,...,...,...
2455,WY,2023,Apr,542,11,2023-04
2456,WY,2023,Feb,1156,23,2023-02
2457,WY,2023,Jan,835,21,2023-01
2458,WY,2023,Mar,1084,18,2023-03


In [21]:
df.to_csv('covid_state_data_20240720.csv', index=False)

In [22]:
len(df)

2460

In [210]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_nums = [1,2,3,4,5,6,7,8,9,10,11,12]
data = {
    'month_index': month_nums,
    'month': months
}
pd.DataFrame(data).to_csv('months.csv', index=False)